Libraries

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout

constraint

In [ ]:
FRAME_WIDTH = 128
FRAME_HEIGHT = 128
FRAMES_PER_VIDEO = 30
NUM_CLASSES = 2  # Violence and Non-Violence
EPOCHS = 10
BATCH_SIZE = 32

data preprocessing mp4

In [ ]:
def preprocess_videos(video_dir):
    X = []
    y = []
    for label, category in enumerate(os.listdir(video_dir)):
        category_path = os.path.join(video_dir, category)
        for video in os.listdir(category_path):
            video_path = os.path.join(category_path, video)
            frames = []
            cap = cv2.VideoCapture(video_path)
            while len(frames) < FRAMES_PER_VIDEO:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
                frames.append(frame)
            cap.release()
            if len(frames) == FRAMES_PER_VIDEO:
                X.append(frames)
                y.append(label)
    return np.array(X), np.array(y)

data preprocessing

In [ ]:
video_dir = '/path/to/your/dataset'
X, y = preprocess_videos(video_dir)

splitting data into train and test func

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

CNN model

In [ ]:
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(FRAMES_PER_VIDEO, FRAME_HEIGHT, FRAME_WIDTH, 3)))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(NUM_CLASSES, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_test, y_test))